# Welcome to ConceptsDreambooth v1.2 Optimized! 

Your refined and continually updated guide to training an artstyle. We're currently using the 1.4 model as it suits our needs best. 

Notebook by Shane Deir (Visit https://www.reddit.com/r/ConceptStream/ for more useful tools in the future)

### Instructions:

Welcome netsculptor! Today is the day you start your journey into the new world of infinite creation. You no longer need a robe and ancient staff to indulge in magic.

I've put together this notebook to give you easy access to the entire process of training your own incredible artstyle from start to finish. 

Since you're here, I'm going to assume that you already have this notebook open on a cloud-instance. (Ignore anything about the cloud if you're training on your own GPU)

Requirements: 

ConceptsDreambooth now works flawlessly on any GPU with at least 24GB of VRAM. 
                                                                                                         
Remember this one rule: 

Have fun and enjoy the process. You're about to create something that humanity has never experienced before. 
                                                                                                  

## Step 1) Build your environment.

Run the cell below (click into it and press the play button, or shift+enter) to install all the dependencies we need.

In [ ]:
# Build your environment
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1


## Step 2) Download the Stable Diffusion 1.4 Model.

You'll need to sign up for an account with Hugging Face and agree to their terms if you haven't already. Run the cell below (If you're on runpod you may get a Javascript error, just refresh the page and it will work), you'll be asked to input an access token. You can find this in your hugging face account settings, then access tokens. Paste it into the box and select login. 

In [ ]:
# Hugging Face Login
from huggingface_hub import notebook_login

notebook_login()

Now that you've authenticated your access token, run the cell below to download the Stable Diffusion 1.4 model.

In [ ]:
# Download the 1.4 sd model
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="CompVis/stable-diffusion-v-1-4-original",
 filename="sd-v1-4.ckpt",
 use_auth_token=True
)

# Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

## Step 3) Regularization Images (These are very important).

Run the cell below to download 1000 pre-generated artstyle regularization images. Downloading these instead of generating them will save you several hours. 

In [1]:
# Download the regularization images.
!git clone https://github.com/ShaneDeir/ConceptStream_regularization_images/


When that's finished run the cell below to move them into the proper directory. 

In [ ]:
# Move the regularization images.
import shutil

original = r'/workspace/ConceptsDreambooth/ConceptStream_regularization_images/'
target = r'/workspace/ConceptsDreambooth/regularization_images'

shutil.move(original, target)

Only use the next two cells if you're experimenting with the amount of regularization images used for training. The cells below are optional and will allow you to generate an additional 1000 artstyle regularization images. Adjust the code accordingly. 

In [ ]:
# Skip this cell and the next one if you downloaded the regularization images.

# Generate 1000 artstyle images.
self_generated_files_prompt = "artstyle" #@param {type:"string"}
self_generated_files_count = 1000 #@param {type:"integer"}

!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter {self_generated_files_count} \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt {self_generated_files_prompt}

dataset=self_generated_files_prompt

!mkdir -p regularization_images/{dataset}
!mv outputs/txt2img-samples/*.png regularization_images/{dataset}

In [ ]:
# Zip up the files for downloading and reuse.
# Download this file locally so you can reuse during another training on this dataset.
!apt-get install -y zip
!zip -r regularization_images.zip regularization_images/{dataset}

## Step 4) Upload your training images.
So here you have 2 options: 

1) You can directly upload a folder named "training_images" filled with your images in:

        /workspace/ConceptsDreambooth/
        
So when you're done uploading you should see all of your training images in:

        /workspace/ConceptsDreambooth/training_images/
        
  OR
  
2) You can upload your images to imgur and follow the instructions in the next two cells.



### CAUTION: 

Be sure to upload an even amount of training images. Otherwise things may fall apart. 

*  Be sure to use at least 30 images with dimensions 512px X 512px. 
*  Use your artistic vision when choosing your training images. 
*  Treat the model with respect.

## Hot Tip

Use https://www.birme.net/?target_width=512&target_height=512 to bulk resize your images to the proper dimensions! 



In [ ]:
# Enter your urls below if you pulling from imgur suits your needs. It's much easier to just upload a folder. 
# This is here so it's easier to hotswap datasets between multiple people for refined testing. 

urls = [
 "https://i.imgur.com/test1.png",
 "https://i.imgur.com/test2.png",
 "https://i.imgur.com/test3.png",
 "https://i.imgur.com/test4.png",
 "https://i.imgur.com/test5.png",
 # Place as many links as you want here in the format displayed above.
 # Make sure they have the .png extension in the url.
]

In [ ]:
# This cell will download your images and place them into a grid.
# The grid is unlikely to work (Which is totally fine) if you have more than 30 images (Which you should). 

import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid

def download_image(url):
 try:
  response = requests.get(url)
 except:
  return None
 return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./training_images"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Step 5) Training Steps & Time and Cost Calculator.

Run the cell below to set your training steps using the optimized formula. (credit to Reddit user u/RealAstropulse for the idea)

In [1]:
# Training steps formula.

import os

# Training images folder path. 
dir_path = r'/workspace/ConceptsDreambooth/training_images/'
# Make sure your training images are in the "/workspace/ConceptsDreambooth/training_images" folder.
training_image_count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        training_image_count += 1

max_training_steps = training_image_count * 50 + 800

print("You have", training_image_count, "training images in your folder. Make sure this is an even number.")

print("Your total max training steps are now set to:", max_training_steps)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/workspace/ConceptsDreambooth/training_images/'

Run the cell below to get an estimate of:

1) How long it will take to train.
2) How much it will cost.

You'll be asked to input your price per hour. If you make a mistake you can always run the cell again. You can also add or remove training images to adjust your time and cost, just be sure to run the cell above again after doing that. 

These estimates are based around training on an Nvidia RTX A6000. As such, they're less likely to be accurate on an RTX 3090. 

In [2]:
# Cost calculation user input.
training_cost = input("How much does your GPU rental cost per hour?: $")
training_cost = float(training_cost)

# Time estimate on an RTX A6000 (2100 derived through testing).
time_estimate = max_training_steps / 2100

# Convert hours in float format into HH:MM format. 
minutes = time_estimate*60
hours, minutes = divmod(minutes, 60)

# Displays the time estimate.
print("Your training time estimate is: ",
      ("%02d:%02d"%(hours,minutes)), "(HH:MM)")

# Displays the cost estimate.
print("The total cost to train this neural network should be around $", round(training_cost * time_estimate, 2),".")

How much does your GPU rental cost per hour?: $ 0.559


NameError: name 'max_training_steps' is not defined

## Step 6) Training.

To train them is your cause! This is the most important step of all so pay close attention. 

In the cell below you'll find a variable named "token", after the "=" replace the word in quotes with the name of your own artstyle. This will be the word you'll use in your prompt to summon your artstyle, so be creative with the spelling. 

You'll also be able to give your project a name! This is just for the filename so you can stay organized. 

Replace the words in the cell below and run the cell to begin training. That's it! You made it! 


In [ ]:
# Training

# Give your project a name! This is only used for the filename, so don't stress if you can't think of something cool.
project_name = "project_name"

# Replace the name inside the quotation marks. This token is the name of your artstyle! Get creative and make sure you choose a unique word that doesn't exist. 
token = "PrimeUniverse"

# Don't change this one. 
class_word = "artstyle" 


reg_data_root = "/workspace/ConceptsDreambooth/regularization_images/artstyle/"

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/ConceptsDreambooth/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

## Copy and name the checkpoint file.

Run the cell below after your model has finished training. 

In [ ]:
# Copy the checkpoint file into our "trained_models" folder.

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use it in your favorite Stable Diffusion GUI or Repo!")

## How to use your trained model!

The way to use your token is `<token> <class>`

So we would say in our prompt "EvenYouniverse artstyle" to summon our trained artstyle. 

Where you place it in your prompt matters.

Try at the start of your prompt to begin with, then reorder with experimentation at your leisure. 

## You made it all the way! 

Now we can run the cell below to generate some images with our trained model! Great moves, keep it up. 
Remember to download your model! 

Be sure to replace "PrimeUniverse" with your own token. 

In [1]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/ConceptsDreambooth/trained_models/{file_name}" \
 --prompt "PrimeUniverse artstyle as Zeus, 8k resolution, intricate details, dramatic lighting, volumetric lighting, cinema 4d render"

Traceback (most recent call last):
  File "C:\GIT\ConceptsDreambooth_2\scripts\stable_txt2img.py", line 2, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


# The Credits

This repo was built upon the shoulders of giants. Let's pay them respect by continuing their work!


Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer: 
https://arxiv.org/abs/2112.10752

Nataniel Ruiz, Yuanzhen Li, Varun Jampani, Yael Pritch, Michael Rubinstein, Kfir Aberman: 
https://arxiv.org/abs/2208.12242


Thank you to everyone involved in the many projects that this repo and Juptyer Notebook borrows from and builds upon. 

You know who you are! 

See You Space Cowboy...